In [1]:
import glob
import pandas as pd
import numpy as np
import sys
import operator
import numpy as np
from scipy import stats
import requests
from IPython.display import HTML
import matplotlib.pylab as plt
import warnings
import geopandas as gpd
from fiona.crs import from_epsg

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
# These are the "Tableau 20" colors as RGB.
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]
 
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.
for i in range(len(tableau20)):
    r, g, b = tableau20[i]
    tableau20[i] = (r / 255., g / 255., b / 255.)

plt.rcParams['figure.figsize'] = 12, 8
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['font.serif'] = 'Arial'
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams['axes.titlesize'] = 22
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['figure.frameon'] = False
# plt.rcParams['axes.prop_cycle'] = cycler('color',tableau20)
plt.rcParams['legend.frameon'] = False
plt.rcParams['legend.fontsize'] = 14

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)


In [3]:
!wget -nc -O data/mappluto_mn.zip https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/mn_mappluto_16v2.zip

# !cd data; unzip -f mappluto_mn.zip
from collections import OrderedDict
plutomap = gpd.GeoDataFrame(gpd.read_file("MNMapPLUTO.shp"))
#pluto['BBL'] = pluto['BBL'].apply(lambda x: x[:-1])
# pluto = pluto.dropna(subset=["CT2010", "CB2010"])
# pluto.crs = {"init": "epsg:2263"}
# pluto = pluto.to_crs({"init": "epsg:4269"})
plutomap.head()

data/mappluto_mn.zip: No such file or directory


,geometry
0,"POLYGON ((973154.7117999941 194614.3315999955,..."
1,"POLYGON ((980492.3622000068 195423.9451999962,..."
2,"POLYGON ((979999.0820000023 196493.2800000012,..."
3,"POLYGON ((980424.4003999978 196274.334800005, ..."
4,"POLYGON ((979561.8711999953 191884.2495999932,..."


In [4]:
pluto = pd.read_csv("MN.csv")
pluto.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (19,22,23,25,63,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,PolicePrct,HealthArea,SanitBoro,SanitDistrict,SanitSub,Address,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,Overlay1,Overlay2,SPDist1,SPDist2,SPDist3,LtdHeight,SplitZone,BldgClass,LandUse,Easements,OwnerType,OwnerName,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,GarageArea,StrgeArea,FactryArea,OtherArea,AreaSource,NumBldgs,NumFloors,UnitsRes,UnitsTotal,LotFront,LotDepth,BldgFront,BldgDepth,Ext,ProxCode,IrrLotCode,LotType,BsmtCode,AssessLand,AssessTot,ExemptLand,ExemptTot,YearBuilt,YearAlter1,YearAlter2,HistDist,Landmark,BuiltFAR,ResidFAR,CommFAR,FacilFAR,BoroCode,BBL,CondoNo,Tract2010,XCoord,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,Version
0,MN,1,10,101,5.0,1018.0,2.0,1.0,10004.0,E007,1.0,8100.0,1.0,1.0,NaN,1 GOVERNORS ISLAND,R3-2,NaN,NaN,NaN,NaN,NaN,GI,NaN,NaN,NaN,N,Y4,8.0,0,P,GOVERNORS ISLAND CORP,7736692,2725731,2725731,0,0,0,0,0,0,2725731,2,158,0.0,0,0,0.0,0.0,0.0,0.0,NaN,0.0,N,5.0,5.0,104445450.0,156510900.0,104445450.0,156510900.0,1900,0,0,Governors Island Historic District,CASTLE WILLIAMS,0.35,0.6,0.0,1.0,1,1.000010e+09,0,5,979071.0,190225.0,16a,Y,199 999,10101.0,NaN,0.0,NaN,1,16V2
1,MN,1,101,101,1.0,1001.0,2.0,1.0,10004.0,E007,1.0,8200.0,1.0,1.0,NaN,1 LIBERTY ISLAND,R3-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,P7,8.0,0,X,U S GOVT LAND & BLDGS,541886,541886,541886,0,0,0,0,0,0,541886,2,10,0.0,0,0,500.0,1046.0,0.0,0.0,NaN,0.0,Y,5.0,5.0,4225950.0,12197250.0,4225950.0,12197250.0,1900,0,0,NaN,STATUE OF LIBERTY NATIONAL MONUMENT,1.00,0.6,0.0,1.0,1,1.000010e+09,0,1,971677.0,190636.0,16a,Y,199 999,10101.0,NaN,0.0,NaN,1,16V2
2,MN,1,201,101,1.0,1000.0,2.0,1.0,10004.0,E007,1.0,8200.0,1.0,1.0,NaN,1 ELLIS ISLAND,R3-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Z9,NaN,0,X,U S GOVT LAND & BLDGS,2764190,603130,603130,0,0,0,0,0,0,603130,2,7,0.0,0,0,0.0,0.0,0.0,0.0,NaN,0.0,N,5.0,5.0,14972400.0,108450450.0,14972400.0,108450450.0,1900,0,0,Ellis Island Historic District,NaN,0.22,0.6,0.0,1.0,1,1.000010e+09,0,1,972790.0,193648.0,12b,NaN,199 999,10101.0,NaN,0.0,NaN,1,16V2
3,MN,1,301,101,NaN,NaN,2.0,1.0,10004.0,E007,1.0,NaN,1.0,1.0,NaN,JOE DIMAGGIO HIGHWAY,ZNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,U0,7.0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,7,0,0.0,0,0,0.0,0.0,0.0,0.0,NaN,0.0,N,0.0,5.0,0.0,0.0,0.0,0.0,0,0,0,NaN,NaN,0.00,0.0,0.0,0.0,1,1.000010e+09,0,0,NaN,NaN,12b,NaN,199 999,10101.0,NaN,0.0,NaN,4,16V2
4,MN,1,401,101,NaN,NaN,2.0,1.0,10004.0,E007,1.0,NaN,1.0,1.0,NaN,JOE DIMAGGIO HIGHWAY,ZNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,U0,7.0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,7,0,0.0,0,0,0.0,0.0,0.0,0.0,NaN,0.0,N,0.0,5.0,0.0,0.0,0.0,0.0,0,0,0,NaN,NaN,0.00,0.0,0.0,0.0,1,1.000010e+09,0,0,NaN,NaN,12b,NaN,1 99 999,10101.0,NaN,0.0,NaN,4,16V2


In [5]:
# ll80_pluto = gpd.GeoDataFrame(df.merge(pluto, left_on="NYC Borough, Block and Lot (BBL)", right_on="BBL"))
# len(ll80_pluto)

## Census data loading starts here

In [6]:
import geopandas as gpd
! unzip -f ACS_2015_5YR_TRACT_36.gdb.zip

acs = gpd.read_file("ACS_2015_5YR_TRACT_36_NEW_YORK.gdb")
acs.head()

Archive:  ACS_2015_5YR_TRACT_36.gdb.zip


,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,Shape_Length,Shape_Area,GEOID_Data,geometry
0,36,093,033102,36093033102,331.02,Census Tract 331.02,G5020,S,183329989.0,3232878.0,+42.7779982,-074.1727018,0.747287,0.020523,14000US36093033102,(POLYGON ((-74.30654999968272 42.7506420000283...
1,36,093,032504,36093032504,325.04,Census Tract 325.04,G5020,S,66000589.0,1265253.0,+42.9010522,-074.0288926,0.411658,0.007415,14000US36093032504,(POLYGON ((-74.09298000022682 42.9558679996085...
2,36,081,004300,36081004300,43,Census Tract 43,G5020,S,170813.0,0.0,+40.7609451,-073.9375143,0.019839,0.000018,14000US36081004300,(POLYGON ((-73.94086199998128 40.7581110004223...
3,36,081,003900,36081003900,39,Census Tract 39,G5020,S,146970.0,0.0,+40.7627994,-073.9409316,0.021572,0.000016,14000US36081003900,(POLYGON ((-73.94330400017941 40.7606580000667...
4,36,081,077908,36081077908,779.08,Census Tract 779.08,G5020,S,200497.0,0.0,+40.7256168,-073.8136929,0.019325,0.000021,14000US36081077908,(POLYGON ((-73.81717100034138 40.7272700000105...


In [7]:
NYS_COUNTIES = set(['005', '047', '061', '081', '085'])
"""
005 - Bronx (Bronx)
047 - Kings (Brooklyn)
061 - New York (Manhattan)
081 - Queens
085 - Richmond (Staten Island)
"""
nyc = acs[acs["COUNTYFP"].isin(NYS_COUNTIES)]
nyc.crs

{'init': u'epsg:4269'}

In [8]:
nyc.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,Shape_Length,Shape_Area,GEOID_Data,geometry
2,36,081,004300,36081004300,43,Census Tract 43,G5020,S,170813.0,0.0,+40.7609451,-073.9375143,0.019839,0.000018,14000US36081004300,(POLYGON ((-73.94086199998128 40.7581110004223...
3,36,081,003900,36081003900,39,Census Tract 39,G5020,S,146970.0,0.0,+40.7627994,-073.9409316,0.021572,0.000016,14000US36081003900,(POLYGON ((-73.94330400017941 40.7606580000667...
4,36,081,077908,36081077908,779.08,Census Tract 779.08,G5020,S,200497.0,0.0,+40.7256168,-073.8136929,0.019325,0.000021,14000US36081077908,(POLYGON ((-73.81717100034138 40.7272700000105...
5,36,081,025700,36081025700,257,Census Tract 257,G5020,S,150587.0,0.0,+40.7501044,-073.9051748,0.023918,0.000016,14000US36081025700,(POLYGON ((-73.90811699989956 40.7461859999839...
6,36,081,077907,36081077907,779.07,Census Tract 779.07,G5020,S,216834.0,0.0,+40.7249623,-073.8184802,0.019465,0.000023,14000US36081077907,(POLYGON ((-73.82172099972433 40.7267080001741...


In [9]:
dog = None
columns_of_interest = set([
    'B25001e1',    #Housing Units
    'B25002e1',    #Occupancy Status
    'B25003e1',    #Tenure
    'B25004e1',    #Vacancy Status
    'B25010e1',   #Average Household Size Of Occupied Housing Units By Tenure
    'B25014e1',    #Tenure By Occupants Per Room
    'B25017e1',   #Rooms
    'B25018e1',   #Median Number Of Rooms
    'B25024e1',   #Units In Structure
    'B25035e1',    #Median Year Structure Built
    'B25041e1',    #Bedrooms
    'B25056e1',   #Contract Rent
    'B25061e1',   #Rent Asked
    'B25063e1', #Gross Rent
    'B25064e1',   #Median Gross Rent (Dollars)
    'B25075e1',   #Value
    'B25077e1',   #Median Value (Dollars)
    'B25081e1',  #Mortgage Status
    'B25085e1',   #Price Asked
    'B25087e1'   #Mortgage Status And Selected Monthly Owner Costs  
    ])
block_level_info = pd.DataFrame()
relevant_fnames = []
for f in glob.glob('ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/*.gdbtable'):
    print ('looking at file {f}.'.format(f=f))
    
    if columns_of_interest == set():
        break
    gdf = gpd.read_file(f)
    used_columns = []
    for column in columns_of_interest:
        if column in gdf.columns:
            relevant_fnames.append(f)
            used_columns.append(column)
            block_level_info[column] = gdf[column]
            if 'GEOID' not in block_level_info.columns:
                block_level_info['GEOID'] = gdf['GEOID']
    [columns_of_interest.remove(c) for c in used_columns]

    del gdf

    
{'ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a00000024.gdbtable'}    

looking at file ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a00000001.gdbtable.
looking at file ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a00000002.gdbtable.
looking at file ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a00000003.gdbtable.
looking at file ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a00000004.gdbtable.
looking at file ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a00000005.gdbtable.
looking at file ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a00000006.gdbtable.
looking at file ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a00000007.gdbtable.
looking at file ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a00000009.gdbtable.
looking at file ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a0000000b.gdbtable.
looking at file ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a0000000c.gdbtable.
looking at file ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a0000000d.gdbtable.
looking at file ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a0000000e.gdbtable.
looking at file ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a0000000f.gdbtable.
looking at file ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a00000010.gdbtable.
lookin

{'ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a00000024.gdbtable'}

In [10]:
block_level_info.head()

,B25014e1,GEOID,B25085e1,B25003e1,B25056e1,B25004e1,B25087e1,B25002e1,B25063e1,B25018e1,B25001e1,B25061e1,B25041e1,B25010e1,B25017e1,B25081e1,B25075e1,B25077e1,B25064e1,B25035e1,B25024e1
0,815,14000US36001000100,0,815,569,138,246,953,569,5.2,953,51,953,2.44,953,246,246,96000.0,829.0,1939.0,953
1,2006,14000US36001000200,58,2006,1481,716,525,2722,1481,4.6,2722,295,2722,2.21,2722,525,525,85300.0,638.0,1939.0,2722
2,2311,14000US36001000300,0,2311,1405,384,906,2695,1405,5.1,2695,245,2695,2.19,2695,906,906,163600.0,954.0,1946.0,2695
3,1037,14000US36001000401,19,1037,420,95,617,1132,420,4.4,1132,76,1132,1.75,1132,617,617,154400.0,1218.0,1986.0,1132
4,2300,14000US36001000403,87,2300,1184,259,1116,2559,1184,5.7,2559,172,2559,2.02,2559,1116,1116,193200.0,955.0,1939.0,2559


In [11]:
# gives back only the unique values from some list
set(relevant_fnames)

{'ACS_2015_5YR_TRACT_36_NEW_YORK.gdb/a00000024.gdbtable'}

In [12]:
block_level_info["GEOID_clean"] = block_level_info["GEOID"].apply(lambda x: x.split("US")[1])
block_census_merged = nyc.merge(block_level_info, left_on="GEOID", right_on="GEOID_clean")

In [13]:
# block_census_merged.to_file('acs.shp')

In [14]:
len(block_census_merged)

2167

In [15]:
# block_census_merged.head()

In [16]:
# rename 

clean_acs = block_census_merged.rename(columns={'B25001e1': 'Housing Units', 'B25002e1': 'Occupancy Status', 'B25004e1': 'Vacancy Status', 'B25010e1': 'Average Household Size Of Occupied Housing Units By Tenure', 'B25014e1': 'Tenure By Occupants Per Room', 'B25017e1': 'Rooms', 'B25018e1': 'Median Number of Rooms', 'B25024e1': 'Units In Structure', 'B25024e1': 'Median Year Structure Built', 'B25041e1': 'Bedrooms', 'B25056e1': 'Contract Rent', 'B25061e1': 'Rent Asked', 'B25063e1': 'Gross Rent','B25064e1': 'Median Gross Rent (Dollars)','B25077e1': 'Value','B25081e1': 'Median Value (Dollars)','B25085e1': 'Mortgage Status', 'B25087e1': 'Mortgage Status And Selected Monthly Owner Costs'})

# columns_of_interest = set([
#     'B25002e1',    #Occupancy Status
#     'B25003e1',    #Tenure
#     'B25004e1',    #Vacancy Status
#     'B25010e1',   #Average Household Size Of Occupied Housing Units By Tenure
#     'B25014e1',    #Tenure By Occupants Per Room
#     'B25017e1',   #Rooms
#     'B25018e1',   #Median Number Of Rooms
#     'B25024e1',   #Units In Structure
#     'B25035e1',    #Median Year Structure Built
#     'B25041e1',    #Bedrooms
#     'B25056e1',   #Contract Rent
#     'B25061e1',   #Rent Asked
#     'B25063e1', #Gross Rent
#     'B25064e1',   #Median Gross Rent (Dollars)
#     'B25075e1',   #Value
#     'B25077e1',   #Median Value (Dollars)
#     'B25081e1',  #Mortgage Status
#     'B25085e1',   #Price Asked
#     'B25087e1'   #Mortgage Status And Selected Monthly Owner Costs  
#     ])

In [17]:
clean_acs.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID_x,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,Shape_Length,Shape_Area,GEOID_Data,geometry,Tenure By Occupants Per Room,GEOID_y,Mortgage Status,B25003e1,Contract Rent,Vacancy Status,Mortgage Status And Selected Monthly Owner Costs,Occupancy Status,Gross Rent,Median Number of Rooms,Housing Units,Rent Asked,Bedrooms,Average Household Size Of Occupied Housing Units By Tenure,Rooms,Median Value (Dollars),B25075e1,Value,Median Gross Rent (Dollars),B25035e1,Median Year Structure Built,GEOID_clean
0,36,081,004300,36081004300,43,Census Tract 43,G5020,S,170813.0,0.0,+40.7609451,-073.9375143,0.019839,0.000018,14000US36081004300,(POLYGON ((-73.94086199998128 40.7581110004223...,1040,14000US36081004300,0,1040,1013,75,27,1115,1013,3.7,1115,0,1115,2.31,1115,27,27,NaN,514.0,1954.0,1115,36081004300
1,36,081,003900,36081003900,39,Census Tract 39,G5020,S,146970.0,0.0,+40.7627994,-073.9409316,0.021572,0.000016,14000US36081003900,(POLYGON ((-73.94330400017941 40.7606580000667...,463,14000US36081003900,10,463,384,60,79,523,384,4.2,523,20,523,3.13,523,79,79,575500.0,1510.0,1947.0,523,36081003900
2,36,081,077908,36081077908,779.08,Census Tract 779.08,G5020,S,200497.0,0.0,+40.7256168,-073.8136929,0.019325,0.000021,14000US36081077908,(POLYGON ((-73.81717100034138 40.7272700000105...,1409,14000US36081077908,0,1409,1298,46,111,1455,1298,3.8,1455,34,1455,2.92,1455,111,111,634400.0,1356.0,1968.0,1455,36081077908
3,36,081,025700,36081025700,257,Census Tract 257,G5020,S,150587.0,0.0,+40.7501044,-073.9051748,0.023918,0.000016,14000US36081025700,(POLYGON ((-73.90811699989956 40.7461859999839...,498,14000US36081025700,0,498,331,62,167,560,331,3.8,560,27,560,2.64,560,167,167,497500.0,1309.0,1947.0,560,36081025700
4,36,081,077907,36081077907,779.07,Census Tract 779.07,G5020,S,216834.0,0.0,+40.7249623,-073.8184802,0.019465,0.000023,14000US36081077907,(POLYGON ((-73.82172099972433 40.7267080001741...,950,14000US36081077907,0,950,734,88,216,1038,734,4.2,1038,51,1038,3.16,1038,216,216,507800.0,1351.0,1955.0,1038,36081077907


In [19]:
clean_acs.to_file('acs1.shp')